# Police Department Incident Data of San Francisco and Chicago - Proposal 
## Eren Tumkaya 


In [2]:
import requests
import numpy as np
import pandas as pd
import zipfile
from io import BytesIO

## About the Project

San Francisco and Chicago are two mega cities in the US, and as in any large city, crime is a significant problem that requires thorough analysis. This project will focus on exploring the relationships between selected features and incident counts in both cities. The ultimate goal is to train efficient machine learning algorithms to explain the variation in crime counts separately for both cities. Afterwards, the results will be compared.

The main datasets are from Chicago's and San Francisco's open data portals. Additionally, other datasets will be utilized to create additional features such as daily and hourly temperature, precipitation, moon status, etc.

Exploring the data will involve utilizing exploratory machine learning models with appropriate parameters, validation techniques, and a variety of statistical tests. 

## The Datasets 

There are 7 datasets I will be using for this project. I will merge some of them to find the answers for my questions. All of them will be ingested via their API's.

### San Franscisco Police Department Incident Reports: 2018 to Present 

This huge dataset includes all the incident reports from 1'st of Januray, 2018 to current date. It can be found [here](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783/about_data), open data portal of San Fransciso Government. Below I will be showing a small part of the dataset and the columns I will be using.

In [9]:
url = "https://data.sfgov.org/resource/wg3w-h783.json"
params = {
    '$select': 'incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,incident_category,incident_subcategory,supervisor_district'}
response = requests.get(url, params=params)
pd.DataFrame(response.json()).head()

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,incident_category,incident_subcategory,supervisor_district
0,2023-03-13T23:41:00.000,2023-03-13T00:00:00.000,23:41,2023,Monday,2023-03-13T23:41:00.000,Recovered Vehicle,Recovered Vehicle,NaN
1,2023-03-01T05:02:00.000,2023-03-01T00:00:00.000,05:02,2023,Wednesday,2023-03-11T15:40:00.000,Larceny Theft,Larceny Theft - Other,NaN
2,2023-03-13T13:16:00.000,2023-03-13T00:00:00.000,13:16,2023,Monday,2023-03-13T13:17:00.000,Recovered Vehicle,Recovered Vehicle,NaN
3,2023-03-13T10:59:00.000,2023-03-13T00:00:00.000,10:59,2023,Monday,2023-03-13T11:00:00.000,Recovered Vehicle,Recovered Vehicle,NaN
4,2023-03-14T18:44:00.000,2023-03-14T00:00:00.000,18:44,2023,Tuesday,2023-03-14T18:45:00.000,Recovered Vehicle,Recovered Vehicle,NaN


* "Date" column will be helpful in merging the dataset with others. It will also give me the "month" info which I will be using for my model.  

* "Incident_time" is a feature which will require me to organize a bit to be able to group the data. I believe I will round it to the nearest whole hour for each respected data point. 

* "Year" can be an indicative feature in my model especially considering the yearly changes in crime numbers. I will have 6 years of data and it would be reasonable to see some changes. 

* "Day of the week" is again a feature that I am curios about seeing how it changes the numbers. I believe it will be a crucial variable in my model as well. 

* I am also planning to use "Incident Category" to see some statistics on how common each type is.


### Weather Condition Datasets for San Francisco
#### Temperature

Finding historical hourly weather data requires a bit of digging for cities. I found what I was looking for in Iowa State University's Environmental Mesonet Portal. It can be found [here](https://mesonet.agron.iastate.edu/request/download.phtml?network=CA_ASOS). Once again, I will be showing a small part of the dataset here. This one has the date and the temperature data from Jan 2018 to today. 

In [12]:
pd.read_csv("https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=SFO&data=tmpc&year1=2018&month1=1&day1=1&year2=2024&month2=4&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3").head()

,station,valid,tmpc
0,SFO,2024-03-31 23:56,13.89
1,SFO,2024-03-31 22:56,15.00
2,SFO,2024-03-31 21:56,16.67
3,SFO,2024-03-31 20:56,17.78
4,SFO,2024-03-31 19:56,17.78


#### Precipitation
I will also ingest the precipitation data from here. Below, I will show it as an example once again. I ingested the data to indicate the values in mm's.

In [13]:
pd.read_csv("https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=SFO&data=p01m&year1=2018&month1=1&day1=1&year2=2024&month2=4&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3").head()

,station,valid,p01m
0,SFO,2024-03-31 23:56,0.00
1,SFO,2024-03-31 22:56,0.00
2,SFO,2024-03-31 21:56,0.00
3,SFO,2024-03-31 20:56,0.00
4,SFO,2024-03-31 19:56,0.00


### Chicago Police Department Incident Reports: 2001 to Present

This dataset includes all incidents from 2001. It has more than 8 million rows. However, since I will be performing a statistical comparasion with this one and the San Franciso dataset, I will only take the data points dating after 2018 into account. Dataset can be found on [this](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/about_data) open portal of Chicago.

In [9]:
url = "https://data.cityofchicago.org/resource/ijzp-q8t2.json"
params = {
    '$select': 'date,year,description,district'}
response = requests.get(url, params=params)
pd.DataFrame(response.json()).head()

,date,year,description,district
0,2007-08-25T09:22:18.000,2007,SIMPLE,024
1,2021-05-24T15:06:00.000,2021,FIRST DEGREE MURDER,025
2,2021-06-26T09:24:00.000,2021,FIRST DEGREE MURDER,017
3,2023-11-09T07:30:00.000,2023,UNLAWFUL ENTRY,019
4,2023-11-12T07:59:00.000,2023,"AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...",006


* I will perform grouping with date after filtering the data. Later I will merge the dataset with other datasets which I found for Chicago.

### Weather Condition Datasets for Chicago
#### Temperature

In [24]:
pd.read_csv("https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=MDW&data=tmpc&year1=2018&month1=1&day1=1&year2=2024&month2=4&day2=4&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3").head()

,station,valid,tmpc
0,MDW,2018-01-01 00:53,-13.28
1,MDW,2018-01-01 01:53,-14.44
2,MDW,2018-01-01 02:53,-15.56
3,MDW,2018-01-01 03:53,-15.56
4,MDW,2018-01-01 04:53,-16.72


* I used the Midway Airport Station Data because it was the closest one to the center.
* The temperatures have units which are set to be "Celcius"
* I must admit that I was a bit surprised when I saw the first few degress. They represent the first day of 2018. I thought I made a mistake while ingesting the data because it was much colder than I expected for the first days. Later I did a bit of research and found [this](https://www.weather.gov/lot/201718newyears_cold#:~:text=New%20Year's%20Eve%20Night%20Fact,high%20temperature%20was%20%2D2%C2%B0.) article. Turns out, those days set some records for being the coldest. I believe my data is correct.

#### Precipitation

In [26]:
pd.read_csv("https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=MDW&data=p01m&year1=2018&month1=1&day1=1&year2=2024&month2=4&day2=4&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3").head()

,station,valid,p01m
0,MDW,2018-01-01 00:53,0.00
1,MDW,2018-01-01 01:53,0.00
2,MDW,2018-01-01 02:53,0.00
3,MDW,2018-01-01 03:53,0.00
4,MDW,2018-01-01 04:53,0.00


### Full Moon Dates 

There were several sources which I could have ingested this data. I prefered to use Astronomical Applications Department
of the U.S. Naval Observatory site. The respected page where I found the data and the info on how to use the API can be found [here](https://aa.usno.navy.mil/data/api#phase).

In [8]:
def fetch_moon_phase_data(start_year, end_year):
    moon_phase_data = []

    for year in range(start_year, end_year + 1):
        url = f"https://aa.usno.navy.mil/api/moon/phases/year?year={year}"
        response = requests.get(url)
        data=pd.DataFrame(response.json()["phasedata"])
        moon_phase_data.append(data)
        
    
    return moon_phase_data

start_year = 2018
end_year = 2024
moon_phase_df = fetch_moon_phase_data(start_year, end_year)

In [12]:
moon_phase_df[0]

,day,month,phase,time,year
0,2,1,Full Moon,02:24,2018
1,8,1,Last Quarter,22:25,2018
2,17,1,New Moon,02:17,2018
3,24,1,First Quarter,22:20,2018
4,31,1,Full Moon,13:27,2018
5,7,2,Last Quarter,15:54,2018
6,15,2,New Moon,21:05,2018
7,23,2,First Quarter,08:09,2018
8,2,3,Full Moon,00:51,2018
9,9,3,Last Quarter,11:20,2018


I will clean this dataset and reduce it to show dates and the "Full Moons" only.

## Main Questions 

###  Is there a significant relation with precipitation and incident counts?

After preparing the datasets correctly, I am planning on creating a new column where I will set it to "1" if there was rain for that hour. I will calculate the hourly averages for the hours with rain and also the hours without rain. I will visualize my findings. Most importantly, I will perform statistical tests to see if I can reject my null hyphothesis.

* Null Hypothesis (H0): The average incident counts during rainy hours and non-rainy hours are equal.  
* Alternative Hypothesis (H1): The average incident counts during rainy hours and non-rainy hours are not equal.  

For this task, it is obvious that I should use "independent samples t-test" since I am comparing two independent group means (hour averages). I will perform this task for both of the cities. Even if there is high relation, it would be interesting to see which one has the lowest p-value and asses the strength of the rejection. 


###  Is there a significant relation with Full Moon and incident counts?

Another task I want to perform is to see whether there is a relation with the days of "Full Moon" and crime counts. To be honest , for this one, I expect my results to be non significant. The way I will perform this task is similar to the previous one, but this time I will group my dataset daily. I will compare the average incident count when there is a Full Moon to the average incident count when there is no Full Moon.  

* Null Hypothesis (H0): The average incident counts for the full moon days and non-full moon days are equal.  
* Alternative Hypothesis (H1): The average incident counts for the full moon days and non-full moon days are not equal.

Independent Samples T-test will be performed to see the significance. Afterwards, I will do this for both of the cities and compare the results.



### Is there a significant relation with Temperature and Incident Counts?

The temperature data was a bit harder for me to decide how to perform the test on. I am planning on grouping by the day and taking the average for each day. It will yield more reliable results than grouping by hour. Then I will end up with with two continous features. I will check the correlation and also build a linear regression model to asses whether there is a relation. * My Null Hypo will be "b0 is equal to zero".

Performing this for both of the cities will show me which city has more relation with temperature for it's crime counts.

## Modeling 

### Preparing the Data

After cleaning the datasets I will add the important features and have a new frame which is ready to be trained. I will give brief info on the features that I want to use.

* The year column   
* The month column  
* The day of the week column 
* The hour column
* The precipitation column
* The Full Moon column
* The temperature column

* The Target: Crime counts

Both datasets will be in a similar format. I will encode the categorical variables before modeling.

### Choosing the ML Algorithm 

Since this is a regression task, there are many algorithms which might be useful. My initial approach will be to try tree based algorithms due to the fact that I have many categorical features which I will encode. I know that they are also good at capturing complex relations. I am planning on starting with a decision tree implementation, then try random forest or gradient boosting to see if they can improve the performance. I will be cautious to address the issue of overfitting when especially trying the latter algorithms. It is important to note that this model will be a predictive one. My main purpose will be to approach the right crime counts for the respected hour.            

Another thing that I want to perform is building a linear regression model from statsmodel library. For this one, my ultimate goal will be to asses the strength of my features. I will create an ANOVA table and draw conclusions. This one will only be for explorative reasons.

### Choosing the Right Validation 

I am planning on doing Monte-Carlo Validation. This will help me form a confidence intervals for my R^2. Also, thanks to shuffling, it will help me getting rid of dependencies in the data which could have a negative effect on my model. Train - Test splitting will be done accordingly. It is important to note that I will try different methods as well.  

### Final Goal

I will build models for both cities individualy and they will give me a clear idea on which features were more important, which were only important for one city and not the other, along with the ones which were not important at all. Moreover, for the predictive model, I will asses the metrics of my regression (R^2, RMSE, MSE) and do a comparision for both model performances.  


